# Results for evaluation with bias 

In [1]:
import os
import numpy as np
import random
import pandas as pd
from sklearn import metrics
from operator import itemgetter
import json

map_dict = {'Male': 0,
            'Female': 1,
            'Sex':2,
            '0-19':3,
            '20-29':4,
            '30-39':5,
            '40-49':6,
            '50-59':7,
            '60-69':8,
            '70-79':9,
            '80-89':10,
            '90-':11,
            'Age': 12,
            'ecig':13,  
            'ex': 14, 
            'never':15,
            'ltOnce':16,
            '1to10':17,
            '21+':18, 
            'Smoke':19,
            'it':20,
            'en':21,
            'es':22,
            'de':23,
            'Others':24 
            }
#Uid;Age;Sex;Medhistory;Smoking;Language;Date;Folder Name;Symptoms;Covid-Tested;Hospitalized;Location;Voice filename;Cough filename;Breath filename 
def get_covid(temp):
    cot, sym, med, smo = temp[9], temp[8], temp[3], temp[4]
    #print(cot, sym, med, smo)
    sym_dict = {'drycough':0.0, 'smelltasteloss':0.0, 'headache':0.0,'sorethroat':0.0,
            'muscleache':0.0,'wetcough':0.0,'shortbreath':0.0,'tightness':0.0,
            'fever':0.0,'dizziness':0.0,'chills':0.0,'runnyblockednose':0.0, 'None': 0.0}
    syms = sym.split(',')
    for s in syms:
        if s == 'tighness':
            s = 'tightness'
        if s == 'drycoough':
            s = 'drycough'
        if s == 'runny':
            s = 'runnyblockednose'
        if s == 'none' or s == '':
            s = 'None'
        if s in sym_dict:
            sym_dict[s] = 1
    sym_feature = [sym_dict[s] for s in sorted(sym_dict)]  
    
    if cot == 'last14' or cot == 'yes' or cot == 'positiveLast14': #or cot == 'positiveOver14' or cot == 'over14' :
       
        if sym in ['None','','none']: #'pnts'
            label = 'covidnosym'
        else:
            label = 'covidsym'

    elif cot == 'negativeNever':
       
        if sym in ['None','','none']:
            label = 'healthnosym'
        else:
            label = 'healthsym'
  
    else:
        label = 'negativeLast14_over14'
    return label,sym_feature

def get_demo(temp):
    dis = [0]*25
    
    uid, age, sex, smo, lan = temp[0], temp[1], temp[2], temp[4], temp[5]
    #print(uid, age, sex, smo, lan)

    if age in map_dict:
        dis[map_dict[age]] = 1.0
    else:
        dis[map_dict['Age']] = 1.0
        
    if sex in map_dict:
        dis[map_dict[sex]] = 1.0
    else:
        dis[map_dict['Sex']] = 1.0               
        
    if smo in map_dict:
        dis[map_dict[smo]] = 1.0
    else:
        dis[map_dict['Smoke']] = 1.0                
        
        #Android 10,IOS 12
    if lan != 'en':  #LOCALE
        if len(uid) == 12:
            if lan not in ['en','it']:
                lan = 'en'
        if len(uid) == 10:
            if lan not in ['en','it','es','de','pt','el','fr','ru','ro','zh','hi']:
                lan = 'en'
        #Web users label manaually 
        if uid in ['2020-04-07-09_13_39_351468',
                 '2020-04-07-18_06_25_599640',
                 '2020-04-09-16_06_44_367357',
                 '2020-04-10-22_47_24_172784',
                 '2020-04-11-06_26_23_961941',
                 '2020-04-11-22_05_49_593805',
                 '2020-04-18-15_48_05_138883',
                 '2020-06-10-05_39_49_549021',
                 '2020-09-23-07_01_51_181332',
                 '2020-11-01-11_42_45_049808',
                 '2020-11-01-12_58_38_378997',
                 '2020-11-03-23_31_39_409588',
                 '2020-11-04-15_18_07_232635',
                 '2020-11-09-23_11_09_661625',
                 '2020-11-13-17_08_23_362377',
                 '2020-11-16-06_11_16_109602',
                 '2020-11-22-11_06_00_777194',
                 '2020-11-26-01_59_23_375517',
                 '2020-12-03-22_29_21_357264',
                 '2020-12-05-17_17_27_367716',
                 '2020-12-05-17_39_04_713242',
                 '2020-12-07-15_23_57_415845',
                 '2020-12-22-18_31_12_168304',
                 '2020-12-25-17_26_41_854217',
                 '2020-12-29-05_57_24_070615',
                 '2020-12-29-07_16_08_949536',
                 '2020-12-29-19_20_29_055459',
                 '2021-01-05-15_13_37_511023',
                 '2021-01-18-13_11_43_410199',
                 '2021-01-30-09_11_50_857102',
                 '2021-03-04-08_23_04_002407',
                 '2021-03-09-14_44_14_876702',
                 '2021-03-09-15_11_42_488832',
                 '2021-03-10-09_53_26_214430',
                 '2021-03-11-16_22_49_221496',
                 '2021-03-20-09_00_53_503347',
                 '']:
            lan = 'en'
        if uid in ['2020-04-07-18_52_05_802580',
                 '2020-04-08-19_50_32_255545',
                 '2020-04-10-09_01_06_580390',
                 '2020-04-13-18_37_12_935759',
                 '2020-04-14-10_12_04_529979',
                 '2020-04-15-07_01_54_598919',
                 '2020-04-15-13_21_10_153301',
                 '2020-04-15-17_14_07_448851',
                 '2020-04-16-11_38_40_431566',
                 '2020-04-17-19_23_25_139694',
                 '2020-04-21-11_36_49_766752',
                 '2020-04-21-12_31_25_239458',
                 '2020-04-26-16_15_50_150762',
                 '2020-04-26-16_20_02_616687',
                 '2020-12-31-08_24_54_055012',
                 '2021-01-13-20_19_47_216023',
                 '2021-02-03-17_24_30_080652',
                 '2021-02-18-18_00_36_137539',
                 '2021-04-22-20_07_08_880185']:
            lan = 'it'
        if uid in ['2020-09-08-14_21_40_763674',
                   '2020-11-11-10_24_29_994922',
                   '2020-11-19-07_35_21_245792',
                   '2020-12-10-23_13_10_824970'
                    ]:
            lan = 'de'
        if uid in ['2020-04-26-10_36_54_607302',
                   '2020-04-26-19_33_46_515975',
                   '2020-05-01-17_53_43_125632',
                   '2020-05-21-08_45_42_105928',
                   '2020-11-04-14_30_24_061857',
                   '2021-02-08-15_57_50_219590',
                   '2021-02-16-21_46_55_000456'
                    ]:
            lan = 'es'
    if lan in map_dict:
        dis[map_dict[lan]] = 1.0
    else:
        dis[map_dict['Others']] = 1.0  
                    
    return dis

demo_dict = {}

with open('../COVID19_prediction/data/preprocess/results_all_raw_0426.csv') as f:
    for index, line in enumerate(f):
        if index>0:
            temp = line.strip().split(';')
            demo_dict[temp[0] + '/' + temp[7]] = temp
demo_label = sorted(map_dict.items(),key = lambda x:x[1],reverse = False) 
            
def get_metrics(probs,label):      
    predicted = []
    for i in range(len(probs)):
        if probs[i]> 0.5:       
            predicted.append(1)
        else:
            predicted.append(0)     

    pre = metrics.precision_score(label, predicted)
    acc = metrics.accuracy_score(label, predicted)
    auc = metrics.roc_auc_score(label, probs)
    precision, recall, _ = metrics.precision_recall_curve(label, probs)
    rec = metrics.recall_score(label, predicted)

    TN, FP, FN, TP = metrics.confusion_matrix(label,predicted).ravel()   
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP*1.0/(TP+FN)    
    # Specificity or true negative rate
    TNR = TN*1.0/(TN+FP) 

    return auc, TPR, TNR, 0

def get_CI(data, AUC, Sen, Spe):
    AUCs = []
    TPRs = []
    TNRs = []
    for s in range(1000):
        np.random.seed(s) #Para2
        sample = np.random.choice(range(len(data)), len(data), replace=True)
        samples = [data[i] for i in sample]
        sample_pro = [x[0] for x in samples]
        sample_label = [x[1] for x in samples]
        try:
            get_metrics(sample_pro,sample_label)
        except ValueError:
            np.random.seed(1001) #Para2
            sample = np.random.choice(range(len(data)), len(data), replace=True)
            samples = [data[i] for i in sample]
            sample_pro = [x[0] for x in samples]
            sample_label = [x[1] for x in samples]
        else:
            auc, TPR, TNR, _ = get_metrics(sample_pro,sample_label)
        AUCs.append(auc)
        TPRs.append(TPR)
        TNRs.append(TNR)
    
    
    q_0 = pd.DataFrame(np.array(AUCs)).quantile(0.025)[0] #2.5% percentile
    q_1 = pd.DataFrame(np.array(AUCs)).quantile(0.975)[0] #97.5% percentile
    
    q_2 = pd.DataFrame(np.array(TPRs)).quantile(0.025)[0] #2.5% percentile
    q_3 = pd.DataFrame(np.array(TPRs)).quantile(0.975)[0] #97.5% percentile

    q_4 = pd.DataFrame(np.array(TNRs)).quantile(0.025)[0] #2.5% percentile
    q_5 = pd.DataFrame(np.array(TNRs)).quantile(0.975)[0] #97.5% percentile
   
    return('&' + str(AUC.round(2)) + '(' + str(q_0.round(2)) + '-' + str(q_1.round(2)) + ')'
           + '&' + str(Sen.round(2)) + '(' + str(q_2.round(2)) + '-' + str(q_3.round(2)) + ')'
             '&' + str(Spe.round(2)) + '(' + str(q_4.round(2)) + '-' + str(q_5.round(2)) + ')' )


## Supplementary Table 2: User Split

In [2]:
File = 'output/main_userSplitS10_2.txt'    

#main results
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('user(samplse)&auc&sensitivity&specificity')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 
# demographic groups 
for d in [0,1]: 
    print(demo_label[d],'-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if demo[d] == 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [9]: 
    print('16-39','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[3:6]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [9]: 
    print('40-59','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[6:8]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [7]: 
    print('60-','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[8:12]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
            
        auc, TPR, TNR,_  = get_metrics(probs,labels)

    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 
    
import pandas as pd
df = pd.read_csv('../COVID19_prediction/data/preprocess/data_0426_en_split.csv', delimiter=',')
fold = df['fold'].tolist()
random.seed(10)  #Para1
random.shuffle(fold)
df['fold_new'] = fold


df_train = df[(df['fold_new']<2) & (df['label']==1)]
df_test = df[(df['fold_new']==2) & (df['label']==1)]
df_train_users = df_train['uid'].tolist()
df_test_users = df_test['uid'].tolist()
train_user = [temp.split('/')[0] for temp in df_train_users]
test_user = [temp.split('/')[0] for temp in df_test_users]
#print(len(set(test_user)),len(test_user),len(set(train_user)), len(train_user))
unseen = []
for u in test_user:
    if u not in train_user:
        unseen.append(u)
#print(len(set(unseen)),len(unseen))

df_train = df[(df['fold_new']<2) & (df['label']==0)]
df_test = df[(df['fold_new']==2) & (df['label']==0)]
df_train_users = df_train['uid'].tolist()
df_test_users = df_test['uid'].tolist()
train_user = [temp.split('/')[0] for temp in df_train_users]
test_user = [temp.split('/')[0] for temp in df_test_users]
#print(len(set(test_user)),len(test_user),len(set(train_user)), len(train_user))
unseen = []
for u in test_user:
    if u not in train_user:
        unseen.append(u)
#print(len(set(unseen)),len(unseen))

df_train = df[(df['fold_new']<2)]
df_test = df[(df['fold_new']==2)]
df_train_users = df_train['uid'].tolist()
df_test_users = df_test['uid'].tolist()
train_user = [temp.split('/')[0] for temp in df_train_users]
test_user = [temp.split('/')[0] for temp in df_test_users]

# sympotom
for d in [' ']: 
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')           
            temp = demo_dict[uid]
            demo = get_demo(temp)
            covid,sym = get_covid(temp)

            if UID in train_user:
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
               
                user.append(UID)
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
            
                
        auc, TPR, TNR,_  = get_metrics(probs,labels)
        
print('Seen:')
ss = get_CI(data,auc, TPR, TNR)
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
      + ss + '//')     
for d in [' ']: 
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')           
            temp = demo_dict[uid]
            demo = get_demo(temp)
            covid,sym = get_covid(temp)

            if UID not in train_user:
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
               
                user.append(UID)
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                #pre = 1 if pro>0.5 else 0
                #if label != pre:
                #    print(uid, pro, label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)
print('Unseen:')
ss = get_CI(data,auc, TPR, TNR)
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
      + ss + '//') 
 

user(samplse)&auc&sensitivity&specificity
&143(161)/139(158)&0.8(0.75-0.84)&0.71(0.63-0.77)&0.74(0.68-0.81)
('Male', 0) -------------------------------------------------------
&76(85)/77(85)&0.81(0.74-0.87)&0.72(0.62-0.81)&0.75(0.66-0.84)//
('Female', 1) -------------------------------------------------------
&65(74)/62(73)&0.79(0.71-0.86)&0.7(0.61-0.81)&0.73(0.62-0.83)//
16-39 -------------------------------------------------------
&69(79)/59(63)&0.82(0.75-0.89)&0.73(0.63-0.83)&0.76(0.65-0.87)//
40-59 -------------------------------------------------------
&59(65)/57(70)&0.76(0.68-0.84)&0.66(0.54-0.77)&0.7(0.59-0.81)//
60- -------------------------------------------------------
&10(12)/15(16)&0.86(0.71-0.98)&0.83(0.58-1.0)&0.88(0.69-1.0)//
Seen:
&123(139)/122(141)&0.8(0.75-0.85)&0.74(0.67-0.81)&0.72(0.64-0.79)//
Unseen:
&20(22)/17(17)&0.8(0.65-0.93)&0.5(0.3-0.71)&0.94(0.8-1.0)//


## Supplementary Table 3: Gender bias

In [3]:
File = 'output/main_gender_bias.txt'    

#main results
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('user(samplse)&auc&sensitivity&specificity')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 
# demographic groups 
for d in [0,1]: 
    print(demo_label[d],'-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if demo[d] == 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [9]: 
    print('16-39','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[3:6]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [9]: 
    print('40-59','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[6:8]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [7]: 
    print('60-','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[8:12]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
            
        auc, TPR, TNR,_  = get_metrics(probs,labels)

    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

user(samplse)&auc&sensitivity&specificity
&100(162)/100(187)&0.75(0.7-0.8)&0.46(0.38-0.54)&0.9(0.86-0.95)
('Male', 0) -------------------------------------------------------
&58(85)/9(18)&0.62(0.49-0.74)&0.66(0.56-0.76)&0.61(0.36-0.83)//
('Female', 1) -------------------------------------------------------
&42(77)/91(169)&0.66(0.58-0.74)&0.23(0.14-0.33)&0.93(0.9-0.97)//
16-39 -------------------------------------------------------
&55(81)/53(83)&0.69(0.61-0.76)&0.41(0.3-0.52)&0.84(0.76-0.92)//
40-59 -------------------------------------------------------
&36(68)/38(86)&0.79(0.71-0.85)&0.56(0.44-0.67)&0.94(0.89-0.99)//
60- -------------------------------------------------------


D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

...

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

&4(8)/7(16)&0.96(0.86-1.0)&0.25(0.0-0.62)&1.0(1.0-1.0)//


D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predic

## Supplementary Table 4 and 5: Age bias

In [4]:
#File = 'output/main_age_bias.txt'    
File = 'output/main_age2_bias.txt'  
#main results
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('user(samplse)&auc&sensitivity&specificity')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 
# demographic groups 
for d in [0,1]: 
    print(demo_label[d],'-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if demo[d] == 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 
'''
for d in [9]: 
    print('16-39','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[3:6]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 
'''
for d in [9]: 
    print('40-59','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[6:8]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)

        auc, TPR, TNR,_  = get_metrics(probs,labels)


    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//') 

for d in [7]: 
    print('60-','-------------------------------------------------------')
    user = []   
    probs = []
    labels = []
    data = []
    negative_user = []
    with open(File) as f:
        for line in f:        
            uid, pro, label = line.split()
            UID, date = uid.split('/')
            
            temp = demo_dict[uid]
            demo = get_demo(temp)
            if sum(demo[8:12]) >= 1:
                user.append(UID)
                pro = float(pro)
                label = float(label)
                label = 1 if label > 0 else 0
                probs.append(pro)
                data.append([pro,label])
                labels.append(label)
                if label == 0:
                    negative_user.append(UID)
            
        auc, TPR, TNR,_  = get_metrics(probs,labels)

    ss = get_CI(data,auc, TPR, TNR)
    print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
          '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'
          + ss + '//')

user(samplse)&auc&sensitivity&specificity
&100(162)/100(162)&0.8(0.74-0.84)&0.71(0.63-0.78)&0.73(0.66-0.79)
('Male', 0) -------------------------------------------------------
&58(85)/58(97)&0.81(0.75-0.87)&0.71(0.61-0.8)&0.77(0.68-0.85)//
('Female', 1) -------------------------------------------------------
&42(77)/42(65)&0.78(0.71-0.85)&0.71(0.61-0.81)&0.66(0.55-0.78)//
40-59 -------------------------------------------------------
&36(68)/72(105)&0.75(0.67-0.83)&0.71(0.6-0.81)&0.7(0.61-0.78)//
60- -------------------------------------------------------
&4(8)/28(57)&0.62(0.45-0.79)&0.25(0.0-0.6)&0.79(0.68-0.89)//


## Supplementary Table 6: Language bias

In [5]:
File = 'output/main_lan_bias.txt'    
# change file to main_lan_B.txt, main_lan_C.txt, main_lan_V.txt
print('user(samplse)&auc&sensitivity&specificity')
#main results
print('Three modality:')
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]==1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('Italy')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]!=1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('English')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 


File = 'output/main_lan_B.txt'    
#main results
print('Breathing modality:')
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]==1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('Italy')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]!=1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('English')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 


File = 'output/main_lan_C.txt'    
#main results
print('Cough modality:')
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]==1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('Italy')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]!=1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('English')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

File = 'output/main_lan_V.txt'    
#main results
print('Voice modality:')
user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if True:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]==1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('Italy')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 

user = []   
probs = []
labels = []
data = []
negative_user = []
with open(File) as f:
    for line in f:        
        uid, pro, label = line.split()
        UID, date = uid.split('/')
        
        temp = demo_dict[uid]
        demo = get_demo(temp)
        if demo[20]!=1:
            user.append(UID)
            pro = float(pro)
            label = float(label)
            label = 1 if label > 0 else 0
            probs.append(pro)
            data.append([pro,label])
            labels.append(label)
            if label == 0:
                negative_user.append(UID)

    auc, TPR, TNR, _   = get_metrics(probs,labels)
ss = get_CI(data, auc, TPR, TNR)
print('English')
print('&'+str(len(set(user))-len(set(negative_user))) + '(' + str(len((user))-len((negative_user))) + ')'
      '/' + str(len(set(negative_user))) + '(' + str(len((negative_user))) + ')'+ ss ) 


user(samplse)&auc&sensitivity&specificity
Three modality:
&100(139)/100(183)&0.77(0.71-0.82)&0.6(0.51-0.68)&0.78(0.72-0.84)
Italy
&48(66)/2(6)&0.42(0.22-0.67)&0.98(0.95-1.0)&0.0(0.0-0.0)
English
&52(73)/98(177)&0.61(0.54-0.69)&0.25(0.15-0.36)&0.81(0.75-0.86)
Breathing modality:
&100(139)/100(183)&0.55(0.49-0.62)&0.56(0.48-0.65)&0.58(0.51-0.66)
Italy
&48(66)/2(6)&0.26(0.06-0.55)&0.62(0.5-0.74)&0.17(0.0-0.57)
English
&52(73)/98(177)&0.53(0.45-0.61)&0.51(0.39-0.63)&0.6(0.53-0.67)
Cough modality:
&100(139)/100(183)&0.64(0.58-0.7)&0.62(0.53-0.7)&0.59(0.51-0.66)
Italy
&48(66)/2(6)&0.57(0.27-0.81)&0.7(0.59-0.81)&0.33(0.0-0.75)
English
&52(73)/98(177)&0.59(0.51-0.68)&0.55(0.44-0.67)&0.6(0.52-0.68)
Voice modality:
&100(139)/100(183)&0.77(0.71-0.82)&0.59(0.5-0.67)&0.82(0.76-0.88)
Italy
&48(66)/2(6)&0.64(0.49-0.77)&1.0(1.0-1.0)&0.0(0.0-0.0)
English
&52(73)/98(177)&0.6(0.52-0.67)&0.22(0.12-0.32)&0.85(0.79-0.9)
